In [ ]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
age_begin = 13
age_end = 100
risk_groups = ['HM', 'HF', 'MSM']
num_risk = 3

all_jur = 10
age_groups = 88
number_of_risk_groups = 3
number_of_compartments = 22
dt = 1/12

num_age = 88
num_comp = number_of_compartments-2

prep_efficiency = 0.99

unaware_index = (1,5,9,13,17)
aware_no_care_index = (2,6,10,14,18)
ART_VLS_index = (3,4,7,8,11,12,15,16,19,20)
VLS_index = (4,8,12,16,20)


pop_growth_rate = 0

gamma = np.array([[0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1]])

scaling_factor_dropout = np.array([[1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0]])

all_jurisdictions = [12011,12031,12057,12086,12095,12099,12103,12]

num_jur = len(all_jurisdictions)

cluster1 = [12011]
cluster2 = [12031]
cluster3 = [12057]
cluster4 = [12086]
cluster5 = [12095]
cluster6 = [12099]
cluster7 = [12103]
cluster8 = [12]

cluster1_index = np.array([0])
cluster2_index = np.array([1])
cluster3_index = np.array([2])
cluster4_index = np.array([3])
cluster5_index = np.array([4])
cluster6_index = np.array([5])
cluster7_index = np.array([6])
cluster8_index = np.array([7])

In [ ]:
theta = 0.3
tau = 0.3
mu = 0.002
alpha = 0.8

c_r_n = 22.13
c_c_n = 10.36
n_n = 0.45
c_add = 52.66

c_r_p = 78.8
c_c_p = 58.91
n_r_p = 10.86
n_c_p = 5.88
c_cnf = 160.07

O_v0 = 16.59
W = 0.2 #[0.1,0.2,0.3]
# del_x = #outreached people

m_cl = 1000
m_nc = 1000
f_c = 56379
f_nc = 64851

m_o = 1000
f_o = 50000

R_art_0 = 235 #[117,235,300]
Y = 0.2#[0.1,0.2,0.3]
m_r = 500
f_r = 22708 #[17977,22708,29330]

# p_art_0 = np.array([0.51041812, 0.5950631 , 0.57486548]) #need for all jur

In [ ]:
x_t_a_0 = np.array([[2.65002951e+03, 2.02175908e+03, 2.62644571e+02],
       [2.93130643e+03, 3.19234512e+03, 2.75230534e+02],
       [2.31156381e+03, 1.87409298e+03, 2.47994235e+02],
       [1.28336037e+03, 1.63841013e+03, 1.82165627e+02],
       [2.28732104e+03, 2.08411730e+03, 2.04998039e+02],
       [2.08080062e+03, 1.77398071e+03, 6.33114791e+01],
       [1.27163355e+03, 8.27325341e+02, 1.10392783e+02],
       [5.38899747e+03, 1.08289578e+04, 8.90588042e+01]])

delta_1 = np.array([[0.02273225, 0.02314549, 0.03582077],
       [0.01738713, 0.02486817, 0.02562606],
       [0.02180753, 0.02303241, 0.03171971],
       [0.01297455, 0.02140446, 0.03777891],
       [0.03267639, 0.03921426, 0.0408384 ],
       [0.02641965, 0.03017813, 0.02270451],
       [0.01969581, 0.01710169, 0.02083178],
       [0.03591706, 0.097678  , 0.03630814]])

p_unaware_0 = np.array([[0.18478 , 0.128639, 0.172811],
       [0.142568, 0.097128, 0.131897],
       [0.178965, 0.125957, 0.17001 ],
       [0.156497, 0.106958, 0.144602],
       [0.152004, 0.105238, 0.142957],
       [0.229819, 0.16207 , 0.214653],
       [0.204284, 0.141858, 0.18907 ],
       [0.1871  , 0.129821, 0.174115]])

p_art_0 = np.array([[0.545374, 0.628153, 0.60888 ],
       [0.598103, 0.674144, 0.66056 ],
       [0.497292, 0.579056, 0.563949],
       [0.596528, 0.674515, 0.657992],
       [0.540639, 0.61927 , 0.606988],
       [0.531104, 0.619656, 0.59218 ],
       [0.571437, 0.656512, 0.631365],
       [0.560562, 0.643504, 0.622747]])

In [ ]:
c_l = 54000

cd4_gt_350_index = (1,2,3,4,5,6,7,8,9,10,11,12)
cd4_200_350_index = (13,14,15,16)
cd_lt_200_index = (17,18,19,20)

QALY_val_gt_350 = 0.935
QALY_val_250_350 = 0.818
QALY_val_lt_200 = 0.702

In [ ]:
def shi_value(I, N):
    return (0.0153/0.0057)*(I/N)

In [ ]:
def cost(data_t_1, data_t, unaware_prop, aware_art_vls_prop, diagnosis_rate_risk, dropout_rate_risk, prep_rate):

    cost_risk = np.zeros((num_jur, number_of_risk_groups))

    for risk in range(len(risk_groups)):
        
        total_pop_t_risk = np.apply_over_axes(np.sum, data_t[:,risk,:,0:21],[1,2]).reshape(num_jur,)
        I_t_1 = np.apply_over_axes(np.sum, data_t_1[:,risk,:,1:21],[1,2]).reshape(num_jur,)
        p_t_1_unaware = unaware_prop[:,risk]
        p_t_1_art_vls = aware_art_vls_prop[:,risk]
        delta_t = diagnosis_rate_risk[:,risk]
        dropout_t = dropout_rate_risk[:,risk]
        r_t_a_risk = delta_t*I_t_1*p_t_1_unaware 
        shi = shi_value(I_t_1, total_pop_t_risk)
        x_t_a_risk = (r_t_a_risk - mu*I_t_1*p_t_1_unaware) / (theta*shi)
        n_t_a_risk = mu*(total_pop_t_risk - I_t_1*p_t_1_unaware - x_t_a_risk) + x_t_a_risk*theta*(1-shi) 
        X_v_risk = tau*c_r_n + (1-tau)*c_c_n + n_n + (1-alpha)*c_add
        Y_v_risk = tau*(c_r_p + n_r_p) + (1-tau)*(c_c_p + n_c_p) + c_cnf + (1-alpha)*c_add
        
        del_x = (x_t_a_risk - x_t_a_0[:,risk])/total_pop_t_risk
        
        O_v = O_v0*np.exp(del_x*W)
        
        X_f_cl_a = ((r_t_a_risk + n_t_a_risk)*alpha / m_cl)*f_c
        X_f_ncl_a = ((r_t_a_risk + n_t_a_risk)*(1 - alpha) / m_nc)*f_nc
        X_f_o_a = (x_t_a_risk / m_o)*f_o

        cost_of_testing = r_t_a_risk*Y_v_risk + x_t_a_risk* O_v + n_t_a_risk*X_v_risk + X_f_cl_a + X_f_ncl_a + X_f_o_a

        d_t_a_risk = (1 - dropout_t)*I_t_1*p_t_1_art_vls
        del_p_art = p_t_1_art_vls - p_art_0[:,risk]
        R_v_risk = R_art_0*np.exp(del_p_art*Y)
        E_f_a = (d_t_a_risk / m_r)*f_r

        cost_retention_in_care = d_t_a_risk*R_v_risk + E_f_a

        if risk == 2:

            prep_adherence_per_person_per_year = 1431
            prep_medication_per_person_per_year = 12599

            prep_cost = np.sum(data_t_1[:,risk,:,0], axis=1)*prep_rate[:,risk]* \
                            (prep_adherence_per_person_per_year + prep_medication_per_person_per_year)

        else:
            prep_cost = 0

        cost_risk[:,risk] = cost_of_testing + cost_retention_in_care + prep_cost
        
    return dt*cost_risk

In [ ]:
def benefit(data_array):
    L_t_risk = np.zeros((num_jur, number_of_risk_groups))

    for risk in range(len(risk_groups)):

        num_uninfected = np.sum(data_array[:,risk,:,0], axis=1)
        num_over_350 = np.apply_over_axes(np.sum, data_array[:,risk,:,cd4_gt_350_index], [0,2]).reshape(num_jur,)
        num_250_350 = np.apply_over_axes(np.sum, data_array[:,risk,:,cd4_200_350_index], [0,2]).reshape(num_jur,)
        num_below_250 = np.apply_over_axes(np.sum, data_array[:,risk,:,cd_lt_200_index], [0,2]).reshape(num_jur,)

        benefit_risk = 1*num_uninfected + QALY_val_gt_350*num_over_350 + QALY_val_250_350*num_250_350 + QALY_val_lt_200*num_below_250

        L_t_risk[:,risk] = benefit_risk
        
    return c_l*dt*L_t_risk